In [1]:
import numpy as np
from numpy import float64 as realN

from overapprox_functionsN import *
from intervalN import *
from interval import and_numpy_int
from numba import jit, typeof, typed

from reachN import *
from reach import *
from DaTaReachControl import generateTraj, synthNextState, synthTraj,\
                                FOverApprox, GOverApprox,Interval

import time
import matplotlib.pyplot as plt
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
# Transformation from Interval type to Tuple(array type)
def n2i(x_lb, x_ub):
    if isinstance(x_lb, int) or isinstance(x_lb, float):
        return Interval(float(x_lb), float(x_ub))
    res = np.full(x_lb.shape, Interval(0),dtype=Interval)
    if len(x_lb.shape) == 1:
        for i in range(res.shape[0]):
            res[i] = Interval(x_lb[i], x_ub[i])
    elif len(x_lb.shape) == 2:
        for i in range(res.shape[0]):
            for j in range(res.shape[1]):
                res[i,j] = Interval(x_lb[i,j], x_ub[i,j])
    else:
        for i in range(res.shape[0]):
            for j in range(res.shape[1]):
                for k in range(res.shape[2]):
                    res[i,j,k] = Interval(x_lb[i,j,k], x_ub[i,j,k])
    return res

def i2n(intVal):
    if isinstance(intVal, Interval):
        return intVal.lb, intVal.ub
    res_lb = np.empty(intVal.shape, dtype=realN)
    res_ub = np.empty(intVal.shape, dtype=realN)
    if len(intVal.shape) == 1:
        for i in range(res_lb.shape[0]):
            res_lb[i], res_ub[i] = intVal[i].lb, intVal[i].ub
    elif len(intVal.shape) == 2:
        for i in range(res_lb.shape[0]):
            for j in range(res_lb.shape[1]):
                res_lb[i,j], res_ub[i,j] = intVal[i,j].lb, intVal[i,j].ub
    else:
        for i in range(res_lb.shape[0]):
            for j in range(res_lb.shape[1]):
                for k in range(res_lb.shape[2]):
                    res_lb[i,j,k], res_ub[i,j,k] = intVal[i,j,k].lb, intVal[i,j,k].ub
    return res_lb, res_ub

def gen_int(shape=None, minVal=-10, widthMax=10):
    if shape is None:
        lb = widthMax * np.random.random() + minVal
        ub = lb + widthMax * np.random.random()
        return Interval(float(lb), float(ub))
    else:
        lb = widthMax * np.random.random(shape) + minVal
        ub = lb + widthMax * np.random.random(shape)
        return n2i(lb,ub)

In [3]:
# Define the Unicycle system dynamics and the initial trajectory is generated
# randomnly from the data

# Define a seed for repeatability
np.random.seed(3338) # 801, 994, 3338
# val = int(np.random.uniform(0,4000))
# print (val)
# np.random.seed(val)

# Sampling time
sampling_time = 0.1

# Define the initial satte and the axis limits
initial_state = np.array([-2, -2.5, np.pi/2])

# Number of data in initial trajectory
n_data_max = 20

# max number of iteration
max_iteration = 70 - n_data_max

################### Input bounds  #################################
v_max = 4
w_max = 0.5 * (2*np.pi)
v_min = -v_max
w_min = -w_max
input_lb = np.array([v_min, w_min])
input_ub = np.array([v_max, w_max])

# Generate input sequence
v_seq = -1 *(np.random.rand(n_data_max,1) - 0) * v_max       # Go only backwards
w_seq = 2 * (np.random.rand(n_data_max,1) - 0.5) * w_max
nSep = int(n_data_max /2)
sepIndexes = np.random.choice([ i for i in range(n_data_max)], nSep, replace=False)
# The trajectory should try system response in each control direction
w_seq[0,0] = 0.0 #
v_seq[0,0] = 0.0
for i in range(1,nSep):
  v_or_theta = np.random.randint(0,2)
  if v_or_theta == 0: # pick v
    w_seq[sepIndexes[i],0] = 0
  else: # pick theta
    v_seq[sepIndexes[i],0] = 0
rand_init_input_vec = np.hstack((v_seq,w_seq))
# print (rand_init_input_vec)
###################################################################

# Generate the random trajectory corresponding to random input sequence
rand_init_traj_vec = np.zeros((n_data_max + 1, initial_state.size))
rand_init_traj_der_vec = np.zeros((n_data_max, initial_state.size))
#######################################################################

# Input signal for the reachable set computation
c_vmax = rand_init_input_vec[-1,0] * -0.5
c_wmax = rand_init_input_vec[-1,1] * 0.3
c_rot = 6
t_end = (n_data_max-1) * sampling_time

@jit(nopython=True, parallel=False, fastmath=True)
def uOver(t_lb, t_ub):
    if t_lb <= t_end+(0.9*sampling_time):
        indx = int(t_lb / sampling_time)
        return rand_init_input_vec[indx,:], rand_init_input_vec[indx,:] 
    x_lb = np.empty(2, dtype=realN)
    x_ub = np.empty(2, dtype=realN)
    x_lb[0] = c_vmax
    x_ub[0] = c_vmax
    x_lb[1], x_ub[1] = cos_i(c_rot*(t_lb-t_end), c_rot*(t_ub-t_end))
    x_lb[1], x_ub[1] = mul_i_scalar(x_lb[1], x_ub[1], c_wmax)
    return x_lb, x_ub

def uOverO(intT):
    x_lb, x_ub = uOver(intT.lb, intT.ub)
    return n2i(x_lb, x_ub).reshape(-1,1)

@jit(nopython=True, parallel=False, fastmath=True)
def uOverDer(t_lb, t_ub):
    x_lb = np.zeros(2, dtype=realN)
    x_ub = np.zeros(2, dtype=realN)
    if t_lb <= t_end+(0.9*sampling_time):
        return x_lb, x_ub
    x_lb[1], x_ub[1] = sin_i(c_rot*(t_lb-t_end), c_rot*(t_ub-t_end))
    x_lb[1], x_ub[1] = mul_i_scalar(x_lb[1], x_ub[1], -c_wmax * c_rot)
    return x_lb, x_ub

def uOverDerO(intT):
    x_lb, x_ub = uOverDer(intT.lb, intT.ub)
    return n2i(x_lb, x_ub).reshape(-1,1)
#######################################################################

########################### Unknown dynamic ###########################
def fFun(x):
    return np.zeros(3)
def GFun(x):
    return np.array([[np.cos(x[2]), 0], [np.sin(x[2]), 0], [0.0, 1.0]])
def uFun(t):
    u_t, _ = uOver(t, t)
    return u_t.reshape(-1,1)
#######################################################################

# Generate a trajectory and use it to compare the over-approximation by
# the new algorithm and the old algorithm
nPoint = 81
dt = 0.05
# Create the time tables
tVal_1 = np.array([ i*dt for i in range(nPoint+1)])
tMeas = np.array([i*sampling_time for i in range(rand_init_traj_der_vec.shape[0])])
# Obtain the trajectory via ode solver
tVal_1, traj, trajDot = synthTraj(fFun, GFun, uFun, initial_state, tVal_1, atol=1e-10, rtol=1e-10)
tVal = tVal_1[:-1]
# New u value for comparison
u_values = np.empty((rand_init_input_vec.shape[1], nPoint))
# Create the u table by first adding the u applied during the trajectory
# used to learn the dynamics
repeat_val = int(sampling_time/dt)
for i in range(u_values.shape[1]):
    t_lb, t_ub = uOver(tVal[i], tVal[i])
    u_values[:,i] = t_lb[:]

# Construct the state evolution and the state derivative evolution
x_values = traj
xdot_values = trajDot[:,:-1]

# Now redefine the state and theur derivative
for i in range(0, repeat_val*rand_init_input_vec.shape[0], repeat_val):
    rand_init_input_vec[int(i/repeat_val),:] = u_values[:,i] 
    rand_init_traj_der_vec[int(i/repeat_val),:] = xdot_values[:,i]
    rand_init_traj_vec[int(i/repeat_val),:] = x_values[:,i]
rand_init_traj_vec[-1,:] = x_values[:,repeat_val*rand_init_input_vec.shape[0]]

In [4]:
# Preview the trajectory
import matplotlib.pyplot as plt
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
plt.style.use('dark_background')
yaxis_label  = ['$\dot{p}_x$', '$\dot{p}_y$', '$\dot{\theta}$']
stateLabel  = ['$p_x$', '$p_y$', '$\theta$']
label_name  = ['$\dot{x}$', '$\mathrm{HC4revise}$', '$\mathrm{ExcitationBased}$']
for i in range(xdot_values.shape[0]):
    plt.figure()
    plt.plot(tVal, xdot_values[i,:], "red", label='$\dot{x}$')
    plt.plot(tMeas, rand_init_traj_der_vec[:,i], 'b*', label='$\mathscr{T}$')
    plt.xlabel('Time (s)')
    plt.ylabel(yaxis_label[i])
    plt.legend(loc='best')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

for i in range(xdot_values.shape[0]):
    plt.figure()
    plt.plot(tVal, x_values[i,:-1], "red", label=stateLabel[i])
    plt.plot(tMeas, rand_init_traj_vec[:-1,i], 'b*', label='$\mathscr{T}$')
    plt.xlabel('Time (s)')
    plt.ylabel(stateLabel[i])
    plt.legend(loc='best')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
Lf = np.array([0, 0, 0], dtype=realN)
LG = np.array([[1,0],[1,0],[0,0]], dtype=realN)
bG = Dict(*depTypebG)
bG[(0,0)] = (-1.0,1.0)
bG[(1,0)] = (-1.0,1.0)

nDepG = Dict(*depTypeG)
nDepG[(0,0)] = np.array([0,1],dtype=np.int64)
nDepG[(1,0)] = np.array([0,1],dtype=np.int64)

@jit(nopython=True)
def knownGfun(x_lb, x_ub):
    res_lb = np.zeros((3,2),dtype=realN)
    res_ub = np.zeros((3,2),dtype=realN)
    res_lb[2,1] = 1
    res_ub[2,1] = 1
    # res_lb[0,0], res_ub[0,0] = cos_i(x_lb[2], x_ub[2])
    return res_lb, res_ub

overApprox = initOverApprox(Lf, LG,  Lfknown=None, LGknown=None, nvDepF=depTypeF,
        nvDepG=nDepG, bf=depTypebf , bG =bG , bGf = depTypeGradF,
        bGG=depTypeGradG, xTraj=rand_init_traj_vec.T,
        xDotTraj=rand_init_traj_der_vec.T, uTraj=rand_init_input_vec.T,
        useGronwall=True, verbose=False, Gknown=knownGfun)

In [7]:
canApproximate(overApprox)

(True, True)

In [6]:
bGx = {}
bGx[(0,0)] = Interval(-1.0,1.0)
bGx[(1,0)] = Interval(-1.0,1.0)
nDepGx = {}
nDepGx[(0,0)] = np.array([0,1],dtype=np.int64)
nDepGx[(1,0)] = np.array([0,1],dtype=np.int64)
knownG = {(2,1) : {-1 : lambda x : 1,
                    0 : lambda x : 0,
                    1 : lambda x : 0,
                    2 : lambda x : 0}}
fOverO = FOverApprox(Lf.reshape(-1,1), traj={'x' : rand_init_traj_vec.T,
                                    'xDot' : rand_init_traj_der_vec.T,
                                    'u' : rand_init_input_vec.T},
                    nDep={}, bf={}, bGf={}, knownFun={},
                    Lknown=None, learnLip=False, verbose=False)
GoverO = GOverApprox(LG, fOverO, traj={'x' : rand_init_traj_vec.T,
                                    'xDot' : rand_init_traj_der_vec.T,
                                    'u' : rand_init_input_vec.T},
                    nDep=nDepGx, bG=bGx, bGG={}, knownFun=knownG,
                    Lknown=None, learnLip=False, verbose=False)

In [8]:
@jit(nopython=True)
def test_n(x, randve):
    for i in range(randve.shape[1]):
        Gover(x, randve[:,i], randve[:,i], knownGfun)

# @jit(nopython=True)
def test_o(x, randve):
    for i in range(randve.shape[1]):
        x(np.array([[randve[j,i]] for j in range(randve.shape[0])]))

def test_inclusion(numb, old, randve):
    for i in range(randve.shape[1]):
        val = Gover(numb, randve[:,i], randve[:,i], knownGfun)
        convVal = n2i(*val)
        val2 = old(np.array([[randve[j,i]] for j in range(randve.shape[0])]))
        for k in range(val2.shape[0]):
            for l in range(val2.shape[1]):
                # print(val2[k,l], convVal[k,l])
                assert val2[k,l].contains(convVal[k,l])


x_min = np.min(rand_init_traj_vec.T, axis=1)
x_max = np.max(rand_init_traj_vec.T, axis=1)
res_x = np.zeros((x_min.shape[0], 20))
for i in range(res_x.shape[1]):
    res_x[:,i] = ((x_max - x_min) * np.random.random() + x_min)[:]
    # print(typeof(res_x[:,i]))

test_inclusion(overApprox, GoverO, res_x)
test_n(overApprox, res_x)

s = time.time()
test_n(overApprox, res_x)
print('Numba : ', time.time()-s)

s = time.time()
test_o(GoverO, res_x)
print('Default : ', time.time()-s)

uVal = gen_int(2, minVal=-0.2, widthMax=0.4)
uN = i2n(uVal)
uVal = uVal.reshape(-1,1)

print(uVal)
dtCoeff = getCoeffGronwall(overApprox, sampling_time, *uN)

r_lb, r_ub = fixpoint(overApprox, res_x[:,0], res_x[:,0], sampling_time, *uN,
             knownf=None, knownG=knownGfun, hOver=None)

r_old = fixpointRecursive(np.array([[res_x[j,0]] for j in range(res_x.shape[0])]),
                sampling_time, uVal, fOverO, GoverO)

r_2 = fixpointGronwall(np.array([[res_x[j,0]] for j in range(res_x.shape[0])]),
                            dtCoeff, uVal, fOverO, GoverO)
print (n2i(r_lb,r_ub))
print(r_old)
print(r_2)

Numba :  0.0009009838104248047
Default :  0.006118297576904297
[[[0.0546397 , 0.3480496]]
 [[0.0106795 , 0.2720523]]]
[[-2.0743704 , -2.0716792] [-2.7593217 , -2.7245168]
 [1.5315239 , 1.5587292]]
[[[-2.0748938 , -2.0730035]]
 [[-2.7593217 , -2.7245168]]
 [[1.5315239 , 1.5587292]]]
[[[-2.1124192 , -2.0363216]]
 [[-2.7973705 , -2.7212729]]
 [[1.4934752 , 1.5695727]]]


In [9]:
# Compute the different over-approximation
@jit(nopython=True, parallel=False, fastmath=True)
def h_numba(approxObj):
    res_lb = np.zeros((x_values.shape[0],nPoint))
    res_ub = np.zeros((x_values.shape[0],nPoint))
    for i in range(nPoint):
        f_lb, f_ub = fover(approxObj, x_values[:,i], x_values[:,i])
        G_lb, G_ub = Gover(approxObj, x_values[:,i], x_values[:,i], knownG=knownGfun)
        res_lb[:,i], res_ub[:,i] = add_i(f_lb, f_ub, *mul_Ms_i(G_lb, G_ub, u_values[:,i]))
    return res_lb, res_ub

def h_old(intX,uVal):
    return fOverO(intX) + np.matmul(GoverO(intX), uVal.reshape(-1,1))  
_ = h_numba(overApprox)

In [10]:
s_time = time.time()
resn_lb, resn_ub =  h_numba(overApprox)
print(time.time() - s_time)
s_time = time.time()
reso = np.full((x_values.shape[0],nPoint), Interval(0))
for i in range(nPoint):
    reso[:,i] = h_old(x_values[:,i].reshape(-1,1), u_values[:,i])[:,0]
reso_lb, reso_ub = i2n(reso)
time.time() - s_time
    # Save the data

0.010094404220581055


0.04052877426147461

In [11]:
import matplotlib.pyplot as plt
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
plt.style.use('dark_background')
yaxis_label  = ['$\dot{p}_x$', '$\dot{p}_y$', '$\theta$']
label_name  = ['$\dot{x}$', '$\mathrm{HC4revise}$', '$\mathrm{ExcitationBased}$']
zoomInd = rand_init_input_vec.shape[0]*repeat_val
for i in range(xdot_values.shape[0]):
    plt.figure()
    plt.plot(tVal, xdot_values[i,:], "red", label='$\dot{x}$')
    plt.plot(tMeas, rand_init_traj_der_vec[:,i], 'b*')
    plt.fill_between(tVal, reso_lb[i,:], reso_ub[i,:], alpha=0.6, facecolor='tab:cyan', label='$\mathrm{ExcitationBased}$')
    plt.fill_between(tVal, resn_lb[i,:], resn_ub[i,:], alpha=0.9, facecolor='tab:green', label='$\mathrm{HC4revise}$')
    plt.xlabel('Time (s)')
    plt.ylabel(yaxis_label[i])
    plt.legend(loc='best')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Discretization for the reachable sets computation
dt_reach = 0.05
@jit(nopython=True, parallel=False, fastmath=True)
def reachSetN(overApprox):
    res_lb = np.empty((x_values.shape[0], x_values.shape[1]+1), dtype=realN)
    res_ub = np.empty((x_values.shape[0], x_values.shape[1]+1), dtype=realN)
    res_lb[:,0] = rand_init_traj_vec[0,:]
    res_ub[:,0] = rand_init_traj_vec[0,:]
    for i in range(rand_init_input_vec.shape[0]):
        curr_x = rand_init_traj_vec[i,:]
        curr_u = rand_init_input_vec[i,:]
        _, x_lb, x_ub = DaTaReachN(overApprox, curr_x, curr_x, tMeas[i], repeat_val, dt, 
                                uOver, uOverDer, knownG=knownGfun)
        for p in range(repeat_val):
            res_lb[:,i*repeat_val+p+1] = x_lb[:,p+1]
            res_ub[:,i*repeat_val+p+1] = x_ub[:,p+1]
    _, x_lb, x_ub = DaTaReachN(overApprox, rand_init_traj_vec[-1,:], rand_init_traj_vec[-1,:], 
                        t_end+sampling_time, x_values.shape[1]-repeat_val*rand_init_input_vec.shape[0], 
                        dt, uOver, uOverDer, knownG=knownGfun)
    for p in range(x_lb.shape[1]-1):
        res_lb[:,repeat_val*rand_init_input_vec.shape[0]+p+1] = x_lb[:,p+1]
        res_ub[:,repeat_val*rand_init_input_vec.shape[0]+p+1] = x_ub[:,p+1]
    return res_lb, res_ub

def reachSetO():
    res_lb = np.full((x_values.shape[0], x_values.shape[1]+1), Interval(0))
    res_lb[:,0] = np.array([Interval(rand_init_traj_vec[0,i]) for i in range(x_values.shape[0])])
    for i in range(rand_init_input_vec.shape[0]):
        curr_x = np.array([Interval(rand_init_traj_vec[i,j]) for j in range(x_values.shape[0])]).reshape(-1,1)
        curr_u = rand_init_input_vec[i,:].reshape(-1,1)
        _, x_lb = DaTaReach(curr_x, tMeas[i], repeat_val, dt, fOverO, GoverO, uOverO, uOverDerO, useFast=True)
        for p in range(repeat_val):
            res_lb[:,i*repeat_val+p+1] = x_lb[:,p+1]
    endPoint = np.array([Interval(rand_init_traj_vec[-1,i]) for i in range(x_values.shape[0])]).reshape(-1,1)
    _, x_lb = DaTaReach(endPoint, t_end+sampling_time, x_values.shape[1]-repeat_val*rand_init_input_vec.shape[0], 
                       dt, fOverO, GoverO, uOverO, uOverDerO, useFast=True)
    for p in range(x_lb.shape[1]-1):
        res_lb[:,repeat_val*rand_init_input_vec.shape[0]+p+1] = x_lb[:,p+1]
    return res_lb
    
_,_ = reachSetN(overApprox)
print('###############################################')
_ = reachSetO()

###############################################


In [14]:
# COmpute the over-approximations
s_time = time.time()
reachn_lb, reachn_ub =  reachSetN(overApprox)
print(time.time() - s_time)

s_time = time.time()
reachO = reachSetO()
reacho_lb, reacho_ub = i2n(reachO)
time.time() - s_time
    # Save the data

0.014092206954956055


0.23214459419250488

In [15]:
# Preview the trajectory
import matplotlib.pyplot as plt
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
plt.style.use('dark_background')
yaxis_label  = ['$\dot{p}_x$', '$\dot{p}_y$', '$\dot{\theta}$']
stateLabel  = ['$p_x$', '$p_y$', '$\theta$']
label_name  = ['$\dot{x}$', '$\mathrm{HC4revise}$', '$\mathrm{ExcitationBased}$']

for i in range(xdot_values.shape[0]):
    plt.figure()
    plt.plot(tVal, x_values[i,:-1], "red", label=stateLabel[i])
    plt.plot(tMeas, rand_init_traj_vec[:-1,i], 'b*', label='$\mathscr{T}$')
    plt.fill_between(tVal, reacho_lb[i,:-2], reacho_ub[i,:-2], alpha=0.6, facecolor='tab:cyan', label='$\mathrm{ExcitationBased}$')
    plt.fill_between(tVal, reachn_lb[i,:-2], reachn_ub[i,:-2], alpha=0.9, facecolor='tab:green', label='$\mathrm{HC4revise}$')
    plt.xlabel('Time (s)')
    plt.ylabel(stateLabel[i])
    plt.legend(loc='best')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# Proceed to the 2-D plot of the trajectory
import matplotlib.pyplot as plt
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
plt.style.use('dark_background')

plt.figure()
plt.plot(x_values[0,:-1], x_values[1,:-1], 'red', label='$\mathrm{Unknown \ trajectory}$')
plt.plot(rand_init_traj_vec[:-1,0], rand_init_traj_vec[:-1,1], 'b*', label='$\mathscr{T}$')

first = True
for i in range(tVal.shape[0]):
    x1,x2,y1,y2 = reacho_lb[0,i], reacho_ub[0,i], reacho_lb[1,i], reacho_ub[1,i]
    if first:
        plt.fill([x1,x2,x2,x1], [y1,y1,y2,y2],\
                facecolor='tab:green', edgecolor='darkgreen', alpha=0.8,\
                label='$\mathrm{HC4revise}$')
        first = False
    plt.fill([x1,x2,x2,x1], [y1,y1,y2,y2],\
        facecolor='tab:green', edgecolor='darkgreen', alpha=0.8)

first = True
for i in range(tVal.shape[0]):
    x1,x2,y1,y2 = reachn_lb[0,i], reachn_ub[0,i], reachn_lb[1,i], reachn_ub[1,i]
    if first:
        plt.fill([x1,x2,x2,x1], [y1,y1,y2,y2],\
                facecolor='tab:cyan', edgecolor='darkcyan', alpha=0.8,\
                label='$\mathrm{ExcitationBased}$')
        first = False
    plt.fill([x1,x2,x2,x1], [y1,y1,y2,y2],\
        facecolor='tab:cyan', edgecolor='darkcyan', alpha=0.8)
    
plt.xlabel('$p_x$')
plt.ylabel('$p_y$')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
U_lb = np.array([-v_max, -w_max])
U_ub = np.array([v_max, w_max])

coeff = getCoeffGronwall(overApprox, dt, U_lb, U_ub)
print(coeff)
b_lb, b_ub, A1_lb, A1_ub, A2_lb, A2_ub = controlAffineOverApprox(overApprox, rand_init_traj_vec[-1,:], dt, U_lb, U_ub, knownG=knownGfun, gronwallCoeff=coeff)

from DaTaReachControl.control import computeAffineLinear
b_val, A_val = computeAffineLinear(rand_init_traj_vec[-1,:].reshape(-1,1), dt, fOverO, GoverO, n2i(U_lb, U_ub).reshape(-1,1), 
                                                    coeff, fixpointWidenCoeff=0.2, zeroDiameter=1e-5, widenZeroInterval=1e-3, gronwall=True)

0.09801960187872605


In [19]:
s_time = time.time()
b_lb, b_ub, A1_lb, A1_ub, A2_lb, A2_ub = controlAffineOverApprox(overApprox, rand_init_traj_vec[-1,:], dt, U_lb, U_ub, knownG=knownGfun, gronwallCoeff=coeff)
print(time.time()-s_time)
s_time = time.time()
b_val, A_val = computeAffineLinear(rand_init_traj_vec[-1,:].reshape(-1,1), dt, fOverO, GoverO, n2i(U_lb, U_ub).reshape(-1,1), 
                                                    coeff, fixpointWidenCoeff=0.2, zeroDiameter=1e-5, widenZeroInterval=1e-3, gronwall=True)
print(time.time()-s_time)

0.0003261566162109375
0.0033469200134277344


In [20]:
print(n2i(b_lb, b_ub))
print(b_val.flatten())

[[-2.5904129 , -2.5904129] [-4.4306366 , -4.4306366]
 [0.9421056 , 0.9421056]]
[[-2.5904129 , -2.5904129] [-4.4306366 , -4.4306366]
 [0.9421056 , 0.9421056]]


In [21]:
print(n2i(A1_lb, A1_ub))
print(A_val[0])

print(n2i(A2_lb, A2_ub))
print(A_val[1])

[[[0.0293201 , 0.0300793] [-0.0050000 , 0.0050000]]
 [[0.0397480 , 0.0406174] [-0.0050000 , 0.0050000]]
 [[0.0000000 , 0.0000000] [0.0500000 , 0.0500000]]]
[[0.0220692 , 0.0300793] [-0.0050000 , 0.0050000]]
[[[0.0253931 , 0.0340063] [0.0000000 , 0.0000000]]
 [[0.0358210 , 0.0445444] [0.0000000 , 0.0000000]]
 [[0.0000000 , 0.0000000] [0.0500000 , 0.0500000]]]
[[0.0386580 , 0.0466681] [-0.0050000 , 0.0050000]]
